# Models

We will cover following models:
* Embedding => Class
* Embedding => Simple RNN => Class
* Embedding => Bi-directional RNN => Class

### Load data
Load the toxic comment classification challenge dataset
and split the dataset into training, validation, testing
#### Input text for training
1. Get the training data
    * read the csv data file using pandas
    * tokenize the data
    * assign a dimension to each word
    * convert into embeddings

#### Read CSV

In [1]:
import pandas as pd
train_csv = './data/toxic-comments/train.csv'
train_df = pd.read_csv(train_csv)
# ToDo : sort the df based on size of comments (no. of words in comment)

In [2]:
rowsums=train_df.iloc[:,2:].sum(axis=1)
train_df['clean']=(rowsums==0)
train_texts = train_df['comment_text']
train_labels = train_df['clean']

#### Output labels for training
2. Get the training label
    * read the labels and convert into one-class labels
    * We will focus on 2 class problem: toxic and non toxic comments
    * We will label all different types of toxic comments into same category of toxic label:
        * 0 for toxic comment
        * 1 for non-toxic comments
    * Later we can explore how to make it multiclass classifier

### Tokenization
Now we have training data in two separate array: an ordered array consisting of comments (input) and another array consisting of class lables in same order (output).

We have to transform this data into network input format and output format.
Steps of preprocessing:

1. Tokenize the text into words
2. Assign each word a dimension


To accompolish step 1 and 2 we will use inbuilt Tokenizer

In [3]:
from keras.preprocessing.text import Tokenizer
max_vocab_size = 10000
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(train_texts)
sequences = tokenizer.texts_to_sequences(train_texts)
print(sequences[0])

word_index = tokenizer.word_index
print('Found %s unique tokes.' % len(word_index))

Using TensorFlow backend.


[688, 75, 1, 126, 130, 177, 29, 672, 4511, 1116, 86, 331, 51, 2278, 50, 6864, 15, 60, 2756, 148, 7, 2937, 34, 117, 1221, 2825, 4, 45, 59, 244, 1, 365, 31, 1, 38, 27, 143, 73, 3462, 89, 3085, 4583, 2273, 985]
Found 210337 unique tokes.


### Batching and Preprocessing (padding) for Embedding
Now once we have the tokens, we will do following steps to create word embeddings  

3. Then use this dimension assignment to define embedding
4. Use word embedding to greate word vector for a comment


We will use a specific type of Layer for this, which is called Embedding Layer. The above generated tokens will go as input to Embedding layer, which will output word embeddings as output to next layer:  

   **Input**: 2D tensor of integers, of shape (samples, sequence_length), where each entry is a sequence of integers (output of above code).  
   **Output**: 3D floating-point tensor of shape (samples, sequence_length, embedding_dimensionality).  

Sequence length can be variable per batch. But in a single batch sequence length will be same for all sequences.  

So from data we have to create batches of sequence of similar length and then pad or truncate each sequence to have same sequence length within a particular batch. And we can use each batch as a training input for embedding layer.  

For sample case: we take 10k sequence from 160k for training in a single batch. And take max sequence length of 50 words.


In [4]:
from keras import preprocessing
training_sequences = sequences[:10000]
training_labels = train_labels[:10000]
seq_max_len = 20
# training padded sequences
train_seq_pad = preprocessing.sequence.pad_sequences(sequences=training_sequences, maxlen=seq_max_len)

# testing padded sequences
testing_sequences = sequences[10000:11000]
testing_labels = train_labels[10000:11000]
test_seq_pad = preprocessing.sequence.pad_sequences(sequences=testing_sequences, maxlen=seq_max_len)

### Model 1. : Embedding to Class

#### Define the model 1
Model 1 is made of 2 layers:
    - Layer 1 is Embedding layer
    - Layer 2 is classification (Dense) Layer

In [5]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.layers.embeddings import Embedding

model_1 = Sequential()

## layer 0: add an embedding layer:
# no. of unique words in the text data, each word in vocab will be assigned an index (dimension).
vocab_size = 10000 
# dimension of word embedding model (output of this layer)
embedding_dim = 8 
# max length of single input data e.g. count of words present in an input sentence, input of this layer
seq_max_len = 20 
# input to layer 0 is data of shape: [batch_size, seq_max_len]
# output of layer 0 is data of shape: [batch_size, embedding_dim, seq_max_len]
model_1.add(Embedding(vocab_size, embedding_dim, input_length=seq_max_len))

## layer 1: flatten the input of shape [batch_size, embedding_dim, seq_max_len] 
#          to output of shape [batch_size, embedding_dimension*seq_max_len]
model_1.add(Flatten())

## layer 2: Dense layer - all nodes from previous layers are connected to each nodes from this layer
#          this has 1 unit/node for classification(toxic/non-toxic)
#          and activation for 2 classes: sigmoind
model_1.add(Dense(1, activation='sigmoid'))

## compile: configure the model for training
# optimizer: it is the method use to update the network, 
#            it is generally variant of stochastic gradient descent (SGD)  
#            this method is use iteratively to update the network weights
# loss:      it is the (objective) function that will be minimised
# metrics:   this is use to measure the performance of network
model_1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

# prints the summary of the model
model_1.summary()

# fit: trains the network for a fixed no. of epoch
history_1 = model_1.fit(train_seq_pad, training_labels, epochs=10, batch_size=32, validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


/home/ravi/code/virtualenvs/deep-learn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 0s 50us/step - loss: 0.4540 - acc: 0.8767 - val_loss: 0.2789 - val_acc: 0.9100
Epoch 2/10
8000/8000 [==============================] - 0s 39us/step - loss: 0.2778 - acc: 0.8955 - val_loss: 0.2431 - val_acc: 0.9125
Epoch 3/10
8000/8000 [==============================] - 0s 39us/step - loss: 0.2335 - acc: 0.9035 - val_loss: 0.2222 - val_acc: 0.9185
Epoch 4/10
8000/8000 [==============================] - 0s 41us/step - loss: 0.2032 - acc: 0.9197 - val_loss: 0.2118 - val_acc: 0.9270
Epoch 5/10
8000/8000 [==============================] - 0s 41us/step - loss: 0.1848 - acc: 0.9289 - val_loss: 0.2084 - val_acc: 0.9315
Epoch 6/10
8000/8000 [==============================] - 0s 41us/step - loss: 0.1729 - acc: 0.9344 - val_loss: 0.2075 - val_acc: 0.9320
Epoch 7/10
8000/8000 [==============================] - 0s 40us/step - loss: 0.1630 - acc: 0.9380 - val_loss: 0.2080 - val_acc: 0.9330
Epoch 8

#### Test the model 1

We will take a small test data from the unused training data to test our basic model.  

`model_1.evaluate` method is use to evaluate the model. For evaluation we give input the test data in the same format as of training data together with label data for the test data to compare with.

Ref: Listing 6.7 Deep Learning with Python book  

In [6]:
print(model_1.metrics_names)
model_1.evaluate(x=test_seq_pad, y=testing_labels)

['loss', 'acc']
1000/1000 [==============================] - 0s 13us/step


[0.20169082522392273, 0.925000011920929]

### Model 2: Embedding => RNN => Output
In this model 2 we will extend the Model 1 by adding an RNN layer in between the Embedding layer and output layer.

#### Define the model 2
Model 2 is made of 3 layers:
    - Layer 1 is Embedding layer
    - Layer 2 is RNN layer
    - Layer 3 is classification (Dense) layer 

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN

# model configurations
vocab_size = 10000
seq_max_len = 20 # this can be removed as it is not required for next layer which is RNN
embedding_dim = 16

# model definition
model_2 = Sequential()
model_2.add(Embedding(vocab_size, embedding_dim, input_length=seq_max_len))
model_2.add(SimpleRNN(32))
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 16)            160000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                1568      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 161,601
Trainable params: 161,601
Non-trainable params: 0
_________________________________________________________________


#### Train the model 2

In [8]:
history_2 = model_2.fit(train_seq_pad, training_labels, epochs=10, batch_size=32, validation_split=0.2)

/home/ravi/code/virtualenvs/deep-learn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 1s 161us/step - loss: 0.3221 - acc: 0.8934 - val_loss: 0.2403 - val_acc: 0.9215
Epoch 2/10
8000/8000 [==============================] - 1s 128us/step - loss: 0.2091 - acc: 0.9239 - val_loss: 0.2113 - val_acc: 0.9295
Epoch 3/10
8000/8000 [==============================] - 1s 130us/step - loss: 0.1545 - acc: 0.9427 - val_loss: 0.2021 - val_acc: 0.9365
Epoch 4/10
8000/8000 [==============================] - 1s 131us/step - loss: 0.1107 - acc: 0.9603 - val_loss: 0.2208 - val_acc: 0.9290
Epoch 5/10
8000/8000 [==============================] - 1s 130us/step - loss: 0.0765 - acc: 0.9729 - val_loss: 0.2169 - val_acc: 0.9370
Epoch 6/10
8000/8000 [==============================] - 1s 130us/step - loss: 0.0506 - acc: 0.9831 - val_loss: 0.2521 - val_acc: 0.9190
Epoch 7/10
8000/8000 [==============================] - 1s 132us/step - loss: 0.0309 - acc: 0.9909 - val_loss: 0.2921 - val_acc: 0.9090


#### Test the model 2

In [9]:
print(model_2.metrics_names)
model_2.evaluate(x=test_seq_pad, y=testing_labels)

['loss', 'acc']
1000/1000 [==============================] - 0s 40us/step


[0.4380330653190613, 0.8820000290870667]

We see that above model didn't have good accuracy compared to much simpler model. We didn't use most of the data, training data is very less and also value of seq_len was less for training data and more for testing data.


We can extend the model by adding more RNN layers in between and for the above we didn't use the out of intermediate output of RNN layer.

#### Extended model 2
Extended model 2 is made of 5 layers:

- Layer 1 is Embedding layer
- Layer 2 is RNN layer (return full sequence)
- Layer 3 is RNN layer (return full sequence)
- Layer 4 is RNN layer (return last output)
- Layer 5 is classification (Dense) layer 


In [10]:
model_2_ext = Sequential()
model_2_ext.add(Embedding(vocab_size, embedding_dim))
# for intermediate layers, we want to return output of each cell of RNN, 
# so that it forms a seq. which is processed by next RNN layer
model_2_ext.add(SimpleRNN(32, return_sequences=True))
model_2_ext.add(SimpleRNN(64, return_sequences=True))
# in final RNN layer we will not return the sequence but only the final output,
# which is use in the next non RNN layer e.g. Dense layer in this case
model_2_ext.add(SimpleRNN(32))
model_2_ext.add(Dense(1, activation='sigmoid'))
model_2_ext.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model_2_ext.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 32)          1568      
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, None, 64)          6208      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 32)                3104      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 170,913
Trainable params: 170,913
Non-trainable params: 0
_________________________________________________________________


#### Train the ext. model 2

In [11]:
history_2_ext = model_2_ext.fit(train_seq_pad, training_labels, epochs=10, batch_size=32, validation_split=0.2)

/home/ravi/code/virtualenvs/deep-learn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 3s 348us/step - loss: 0.3289 - acc: 0.8925 - val_loss: 0.2502 - val_acc: 0.9170
Epoch 2/10
8000/8000 [==============================] - 2s 250us/step - loss: 0.2080 - acc: 0.9266 - val_loss: 0.2127 - val_acc: 0.9305
Epoch 3/10
8000/8000 [==============================] - 2s 250us/step - loss: 0.1327 - acc: 0.9546 - val_loss: 0.2322 - val_acc: 0.9260
Epoch 4/10
8000/8000 [==============================] - 2s 250us/step - loss: 0.0809 - acc: 0.9734 - val_loss: 0.2839 - val_acc: 0.9110
Epoch 5/10
8000/8000 [==============================] - 2s 253us/step - loss: 0.0428 - acc: 0.9858 - val_loss: 0.3638 - val_acc: 0.8960
Epoch 6/10
8000/8000 [==============================] - 2s 263us/step - loss: 0.0228 - acc: 0.9921 - val_loss: 0.4055 - val_acc: 0.8980
Epoch 7/10
8000/8000 [==============================] - 2s 257us/step - loss: 0.0123 - acc: 0.9959 - val_loss: 0.5112 - val_acc: 0.8825


#### Test the ext. model 2

In [12]:
print(model_2_ext.metrics_names)
model_2_ext.evaluate(x=test_seq_pad, y=testing_labels)

['loss', 'acc']
1000/1000 [==============================] - 0s 82us/step


[0.7782186584472657, 0.8790000081062317]

### Model 3: Embedding => Bidirectional RNN => Output
In this model 3 we will extend the Model 2 by wrapping the RNN layer with a Bidirectional wrapper.

#### Define the model 3
Extended model 3 is made of 3 layers:

- Layer 1 is Embedding layer
- Layer 2 is Bidirectional RNN layer (return last output)
- Layer 3 is classification (Dense) layer 

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN
from keras.layers.wrappers import Bidirectional

# model configurations
vocab_size = 10000
seq_max_len = 20 # this can be removed as it is not required for next layer which is RNN
embedding_dim = 16

# model definition
model_3 = Sequential()
model_3.add(Embedding(vocab_size, embedding_dim, input_length=seq_max_len))
# [1] This will create two copies of the hidden layer, 
# one fit in the input sequences as-is and one on a reversed copy of the input sequence. 
# By default, the output values from these LSTMs will be concatenated.
model_3.add(Bidirectional(SimpleRNN(32)))
model_3.add(Dense(1, activation='sigmoid'))
model_3.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model_3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 16)            160000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                3136      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 163,201
Trainable params: 163,201
Non-trainable params: 0
_________________________________________________________________


#### Train model 3

In [14]:
history_3 = model_3.fit(train_seq_pad, training_labels, epochs=10, batch_size=32, validation_split=0.2)

/home/ravi/code/virtualenvs/deep-learn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 2s 229us/step - loss: 0.3031 - acc: 0.8986 - val_loss: 0.2260 - val_acc: 0.9225
Epoch 2/10
8000/8000 [==============================] - 2s 193us/step - loss: 0.2001 - acc: 0.9320 - val_loss: 0.1998 - val_acc: 0.9360
Epoch 3/10
8000/8000 [==============================] - 2s 198us/step - loss: 0.1465 - acc: 0.9504 - val_loss: 0.2050 - val_acc: 0.9270
Epoch 4/10
8000/8000 [==============================] - 2s 189us/step - loss: 0.1048 - acc: 0.9641 - val_loss: 0.2162 - val_acc: 0.9325
Epoch 5/10
8000/8000 [==============================] - 2s 203us/step - loss: 0.0716 - acc: 0.9759 - val_loss: 0.2518 - val_acc: 0.9160
Epoch 6/10
8000/8000 [==============================] - 2s 204us/step - loss: 0.0424 - acc: 0.9866 - val_loss: 0.2988 - val_acc: 0.9085
Epoch 7/10
8000/8000 [==============================] - 2s 200us/step - loss: 0.0271 - acc: 0.9925 - val_loss: 0.2838 - val_acc: 0.9285


#### Testing model 3

In [15]:
print(model_3.metrics_names)
model_3.evaluate(x=test_seq_pad, y=testing_labels)

['loss', 'acc']
1000/1000 [==============================] - 0s 58us/step


[0.4578404839038849, 0.8730000257492065]

Similarly like model 2, model 3 can be extended by adding more bidirectional layers in between.  

#### Extended model 3
Extended model 3 is made of 5 layers:

- Layer 1 is Embedding layer
- Layer 2 is Bidirectional RNN layer (return full sequence)
- Layer 3 is Bidirectional RNN layer (return full sequence)
- Layer 4 is Bidirectional RNN layer (return last output)
- Layer 5 is classification (Dense) layer 


In [16]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN
from keras.layers.wrappers import Bidirectional

# model configurations
vocab_size = 10000
seq_max_len = 20 # this can be removed as it is not required for next layer which is RNN
embedding_dim = 16

In [17]:
# model definition
model_3_ext = Sequential()
model_3_ext.add(Embedding(vocab_size, embedding_dim, input_length=seq_max_len))
model_3_ext.add(Bidirectional(SimpleRNN(32, return_sequences=True)))
model_3_ext.add(Bidirectional(SimpleRNN(64, return_sequences=True)))
model_3_ext.add(Bidirectional(SimpleRNN(32)))
model_3_ext.add(Dense(1, activation='sigmoid'))
model_3_ext.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model_3_ext.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 20, 16)            160000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 20, 64)            3136      
_________________________________________________________________
bidirectional_3 (Bidirection (None, 20, 128)           16512     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                10304     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 190,017
Trainable params: 190,017
Non-trainable params: 0
_________________________________________________________________


#### Train ext. model 3

In [18]:
history_3_ext = model_3_ext.fit(train_seq_pad, training_labels, epochs=10, batch_size=32, validation_split=0.2)

/home/ravi/code/virtualenvs/deep-learn/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 5s 587us/step - loss: 0.3033 - acc: 0.8999 - val_loss: 0.2239 - val_acc: 0.9260
Epoch 2/10
8000/8000 [==============================] - 4s 462us/step - loss: 0.1896 - acc: 0.9352 - val_loss: 0.2331 - val_acc: 0.9285
Epoch 3/10
8000/8000 [==============================] - 3s 424us/step - loss: 0.1191 - acc: 0.9599 - val_loss: 0.2363 - val_acc: 0.9290
Epoch 4/10
8000/8000 [==============================] - 4s 439us/step - loss: 0.0698 - acc: 0.9768 - val_loss: 0.2847 - val_acc: 0.9250
Epoch 5/10
8000/8000 [==============================] - 3s 419us/step - loss: 0.0413 - acc: 0.9879 - val_loss: 0.3350 - val_acc: 0.9110
Epoch 6/10
8000/8000 [==============================] - 3s 416us/step - loss: 0.0236 - acc: 0.9930 - val_loss: 0.3737 - val_acc: 0.9065
Epoch 7/10
8000/8000 [==============================] - 3s 417us/step - loss: 0.0143 - acc: 0.9954 - val_loss: 0.5692 - val_acc: 0.8740


#### Test ext. model 3

In [19]:
print(model_3_ext.metrics_names)
model_3_ext.evaluate(x=test_seq_pad, y=testing_labels)

['loss', 'acc']
1000/1000 [==============================] - 0s 144us/step


[0.8108287644386292, 0.8569999933242798]

### Plotting the above results

//ToDo: train the above m


In [20]:
import matplotlib.pyplot

##### Ref.:
1. https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/
